In this project I will use Unsuprvised model (Isolation forest) and Supervised model (choosen later).

Isolation forest needs all the normal data and not labeled. 

Supervised will need only the anomlies.

In [2]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os

# --- 1. Connect to PostgreSQL ---
load_dotenv()

user = os.getenv("POSTGRES_USER")
password = os.getenv("POSTGRES_PASSWORD")
host = os.getenv("POSTGRES_HOST")
port = os.getenv("POSTGRES_PORT")
db = os.getenv("POSTGRES_DB")

engine = create_engine(f"postgresql://{user}:{password}@{host}:{port}/{db}")
connection = engine.connect()

# --- 2. Load only NORMAL traffic for Isolation Forest ---
query_normal = """
SELECT *
FROM traffic
WHERE label = 'normal'
"""
df_normal = pd.read_sql(query_normal, connection)

# --- 3. Identify feature types ---
categorical_cols = ['protocol_type', 'service', 'flag']
numerical_cols = df_normal.drop(columns=['label', 'difficulty_level'] + categorical_cols).columns.tolist()

# --- 4. Build preprocessing pipeline ---
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols),
        ('cat', OneHotEncoder(sparse_output=False, drop='first'), categorical_cols)
    ]
)

# --- 5. Apply transformations ---
X_normal = preprocessor.fit_transform(df_normal)

# --- 6. Convert back to DataFrame for inspection or saving ---
# Get one-hot column names
ohe_cols = preprocessor.named_transformers_['cat'].get_feature_names_out(categorical_cols)
all_cols = numerical_cols + list(ohe_cols)
df_normal_processed = pd.DataFrame(X_normal, columns=all_cols)

# --- 7. Save for later ---
df_normal_processed.to_csv("../data/isolation_forest_train.csv", index=False)
print("Isolation Forest training dataset ready and saved!")

Isolation Forest training dataset ready and saved!


Use grouping in Supervised model data

In [7]:
import pandas as pd
from sqlalchemy import create_engine
import os
from dotenv import load_dotenv

# Load .env
load_dotenv()

# DB connection variables
user = os.getenv("POSTGRES_USER")
password = os.getenv("POSTGRES_PASSWORD")
host = os.getenv("POSTGRES_HOST")
port = os.getenv("POSTGRES_PORT")
db = os.getenv("POSTGRES_DB")

# Connect
engine = create_engine(f"postgresql://{user}:{password}@{host}:{port}/{db}")
connection = engine.connect()

# --- Query only anomaly rows ---
query_anomalies = """
SELECT *
FROM traffic
WHERE label != 'normal'
"""
df_anomalies = pd.read_sql(query_anomalies, connection)

# --- Map attack groups ---
attack_groups = {
    'DoS': ['neptune', 'smurf', 'teardrop', 'pod', 'land'],
    'Probe': ['satan', 'ipsweep', 'portsweep', 'nmap'],
    'R2L': ['guess_passwd', 'ftp_write', 'imap', 'phf', 'warezclient', 'multihop', 'warezmaster'],
    'U2R': ['buffer_overflow', 'loadmodule', 'rootkit', 'perl', 'spy']
}

def map_attack_group(label):
    for group, attacks in attack_groups.items():
        if label in attacks:
            return group
    return 'Other'  # fallback, just in case

# Apply mapping
df_anomalies['attack_group'] = df_anomalies['label'].apply(map_attack_group)

# Check counts
print(df_anomalies['attack_group'].value_counts())
import os
os.makedirs('../results', exist_ok=True)

# Save df_anomalies to CSV
df_anomalies.to_csv('../data/anomalies_grouped.csv', index=False)
print("Anomalies saved to CSV!")

attack_group
DoS      31480
Probe     8159
R2L        696
Other      669
U2R         37
Name: count, dtype: int64
Anomalies saved to CSV!


Downsampling

In [19]:
import pandas as pd

# Check counts
print(df_anomalies['attack_group'].value_counts())

# Separate each group
dos = df_anomalies[df_anomalies['attack_group'] == 'DoS']
probe = df_anomalies[df_anomalies['attack_group'] == 'Probe']
r2l = df_anomalies[df_anomalies['attack_group'] == 'R2L']
u2r = df_anomalies[df_anomalies['attack_group'] == 'U2R']
other = df_anomalies[df_anomalies['attack_group'] == 'Other']

# Decide on a target number per class (for example, match Probe's size ~8k)
target_count = 11480

# Downsample DoS
dos_downsampled = dos.sample(n=target_count, random_state=42)

# Downsample Other (optional, to keep small)
#probe_downsampled = probe.sample(n=target_count, random_state=42)

# Keep the smaller classes as-is (Probe, R2L, U2R)
df_balanced = pd.concat([dos_downsampled, probe, r2l, u2r], axis=0)

# Shuffle the dataset
df_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

# Check new distribution
print(df_balanced['attack_group'].value_counts())

output_path = '../data/balanced_anomalies.csv'  
df_balanced.to_csv(output_path, index=False)

print(f"Balanced dataset saved to {output_path}")

attack_group
DoS      31480
Probe     8159
R2L        696
Other      669
U2R         37
Name: count, dtype: int64
attack_group
DoS      11480
Probe     8159
R2L        696
U2R         37
Name: count, dtype: int64
Balanced dataset saved to ../data/balanced_anomalies.csv
